# How to work with pp_network.py

*pp_network.py* has a class called *net_model* that handles the whole network side of our simulation. The network state object includes a pandapower object called *net* as well as methods to implement actions, calculate the reward, and run simulations on *net*.

Another matpower note: net.line has an attribute 'in_service' so it should be easy for us to simulate a fallen line


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks
import pandapower.plotting
import pp_network
from pp_network import *

## Initialize network

Options to initialize: 'case5','case9','case14','rural_1','rural_2','village_1','village_2','suburb_1','iceland',etc... Basically pick anything from https://pandapower.readthedocs.io/en/v1.6.0/networks.html

In [2]:
network_model = NetModel(network_name='rural_1', zero_out_gen_shunt_storage=True)

In [3]:
print('Number of load buses: ',network_model.net.load.shape[0])
print('Which buses are they on: ',network_model.net.load.bus.values)

Number of load buses:  14
Which buses are they on:  [ 3  8  9 10 11 19 20 21 22 23 24 25  7 13]


In [4]:
network_model.net.sgen

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
0,None,8,0,0,NaN,1.0,True,None
1,None,9,0,0,NaN,1.0,True,None
2,None,11,0,0,NaN,1.0,True,None
3,None,22,0,0,NaN,1.0,True,None
4,None,23,0,0,NaN,1.0,True,None


### Look at powerflows to begin with: 

In [5]:
network_model.run_powerflow()

In [6]:
network_model.net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-78.80322,-1.780376
1,0.995039,-1.090849,0.00000,0.000000
2,0.993303,-1.129337,0.00000,0.000000
3,0.992705,-1.133796,5.10000,0.000000
4,0.990056,-1.200407,0.00000,0.000000
5,0.985967,-1.291037,0.00000,0.000000
6,0.982775,-1.362257,0.00000,0.000000
7,0.980482,-1.413681,7.90000,0.000000
8,0.989456,-1.204895,5.10000,0.000000
9,0.985365,-1.295563,5.10000,0.000000


In [7]:
network_model.net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,5.111992,0.000155,-5.103071,-9.521132e-05,0.008921,0.000060,0.007415,0.007415,0.007415,2.746411
1,5.103071,0.000095,-5.100000,-1.862321e-12,0.003071,0.000095,0.007415,0.007415,0.007415,5.222050
2,28.641596,0.120211,-28.498339,-6.683071e-02,0.143257,0.053380,0.041547,0.041547,0.041547,15.387799
3,23.395247,0.066731,-23.298701,-3.130114e-02,0.096546,0.035430,0.034107,0.034107,0.034107,12.632382
4,18.195584,0.031196,-18.136699,-1.023829e-02,0.058885,0.020957,0.026637,0.026637,0.026637,9.865516
5,13.033562,0.010128,-13.003152,-1.134231e-04,0.030410,0.010015,0.019142,0.019142,0.019142,7.089650
6,5.103091,0.000100,-5.100000,-4.922752e-12,0.003091,0.000100,0.007440,0.007440,0.007440,5.239195
7,5.103117,0.000106,-5.100000,-1.035662e-11,0.003117,0.000106,0.007471,0.007471,0.007471,5.260949
8,5.103137,0.000110,-5.100000,-1.614929e-11,0.003137,0.000110,0.007495,0.007495,0.007495,5.278058
9,5.103152,0.000113,-5.100000,-2.092239e-11,0.003152,0.000113,0.007512,0.007512,0.007512,5.290419


In [8]:
network_model.net.res_sgen # we set these to zero as an option in the instantiation

,p_kw,q_kvar
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


### Fake test data

In [9]:
num_times = 24
p_load_data = np.random.randn(network_model.net.load.shape[0],num_times)
q_load_data = np.random.randn(network_model.net.load.shape[0],num_times)

## Add generators

In [10]:
# Add static generators at bus 7 and at bus 10 and initialize their real power production as 1 kW
network_model.add_sgen(7,1.0) 
network_model.add_sgen(10,1.0)

In [11]:
network_model.net.sgen

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
0,None,8,0,0,NaN,1.0,True,None
1,None,9,0,0,NaN,1.0,True,None
2,None,11,0,0,NaN,1.0,True,None
3,None,22,0,0,NaN,1.0,True,None
4,None,23,0,0,NaN,1.0,True,None
5,None,7,1,0,NaN,1.0,True,None
6,None,10,1,0,NaN,1.0,True,None


# Add battery

In [12]:
# Initialize at bus 3 with power flow 1.0 (charging), capacity 10 kWh, initialy SOC 0 %
network_model.add_battery(3,1.0,10,0.0) 

In [13]:
# How many batteries?
network_model.net.storage.shape[0]

1

In [14]:
network_model.net.storage

,name,bus,p_kw,q_kvar,sn_kva,soc_percent,min_e_kwh,max_e_kwh,scaling,in_service,type
0,None,3,1.0,0.0,NaN,0.0,0.0,10.0,1.0,True,None


### Change the battery power

In [15]:
network_model.update_batteries(battery_powers=2*np.ones((network_model.net.storage.shape[0],)), dt=1)
network_model.net.storage

,name,bus,p_kw,q_kvar,sn_kva,soc_percent,min_e_kwh,max_e_kwh,scaling,in_service,type
0,None,3,2.0,0.0,NaN,0.2,0.0,10.0,1.0,True,None


## Update loads and run a powerflow

In [16]:
network_model.update_loads(p_load_data[:,0],q_load_data[:,0])

In [17]:
network_model.run_powerflow()

In [18]:
network_model.net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,1.011479,1.305344,-1.010555,-1.308394,0.000924,-0.003050,0.002385,0.002389,0.002389,0.884744
1,1.010555,1.308394,-1.010236,-1.308658,0.000319,-0.000264,0.002389,0.002389,0.002389,1.682506
2,2.972627,-0.359798,-2.971075,0.358653,0.001552,-0.001145,0.004324,0.004324,0.004324,1.601639
3,3.858535,0.509577,-3.855910,-0.510307,0.002625,-0.000730,0.005624,0.005624,0.005624,2.082965
4,4.500971,0.224713,-4.497447,-0.225095,0.003524,-0.000382,0.006516,0.006516,0.006516,2.413458
5,1.933044,-0.399952,-1.932367,0.398477,0.000677,-0.001475,0.002857,0.002856,0.002857,1.057973
6,-0.887460,-0.868230,0.887640,0.867948,0.000180,-0.000282,0.001794,0.001794,0.001794,1.263321
7,-0.645061,0.285595,0.645119,-0.285892,0.000058,-0.000298,0.001020,0.001020,0.001020,0.718465
8,2.564403,0.625046,-2.563588,-0.625245,0.000815,-0.000199,0.003820,0.003820,0.003820,2.689898
9,0.873087,-0.063021,-0.872998,0.062728,0.000090,-0.000293,0.001267,0.001267,0.001267,0.892332


# Calculate the Reward

In [19]:
network_model.calculate_reward()
network_model.reward_val

0.0

# Key Documentation: 

https://pandapower.readthedocs.io/en/v1.6.0/elements.html
